In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
import torchvision.models as models
import torch
import torch.nn as nn

In [3]:
densenet = models.densenet121(pretrained=False)

In [19]:
path = "/home/pdpino/workspace-medical-ai/public_checkpoints/cls-0321_052008_chexpert_densenet-121-v2_drop0.3_labels13_normS_wd1e-05_size256_lr0.0001_sch-roc_auc-p3-f0.5_shuffle.pt"
checkpoint = torch.load(path, map_location='cuda')

In [30]:
del checkpoint["prediction.weight"]
del checkpoint["prediction.bias"]

In [27]:
densenet.classifier = None

In [31]:
densenet.load_state_dict(checkpoint)

<All keys matched successfully>

In [5]:
BS = 5 # batch_size
images = torch.rand(BS, 3, 256, 256)

In [8]:
densenet(images).shape

torch.Size([5, 1000])

In [12]:
image_encoder = nn.Sequential(*list(densenet.children())[:-1])

In [9]:
densenet.features(images).shape

torch.Size([5, 1024, 8, 8])

In [13]:
image_encoder(images).shape

torch.Size([5, 1024, 8, 8])

In [1]:
from medvqa.datasets.medical_tags_extractor import MedicalTagsExtractor

In [2]:
med_tags_extractor = MedicalTagsExtractor('medical_terms_frequency__20220317_145545.pkl')

In [4]:
_text = 'small right and moderate left pleural effusions are improving revealing stable moderate size consolidation at the right lung base that could be atelectasis or pneumonia and improved left lower lobe atelectasis. Heart is probably moderately enlarged so substantially obscured by pleural effusion and atelectasis'
list1 = med_tags_extractor.extract_tags(_text)
list2 = [med_tags_extractor.tags[x] for x in med_tags_extractor.extract_tag_ids(_text)]
assert sorted(list1) == sorted(list2)
print(list1)
print(list2)

['lung', 'obscured', 'effusion', 'consolidation', 'pneumonia', 'base', 'moderate', 'size', 'atelectasis', 'heart', 'lobe', 'small', 'right', 'enlarged', 'left', 'pleural', 'lower']
['lung', 'obscured', 'effusion', 'consolidation', 'pneumonia', 'base', 'moderate', 'size', 'atelectasis', 'heart', 'lobe', 'small', 'right', 'enlarged', 'left', 'pleural', 'lower']


In [5]:
len(med_tags_extractor.tags)

1956

In [9]:
from sklearn.metrics import accuracy_score

In [26]:
torch.rand(9) - 0.5

tensor([-0.0922,  0.4901,  0.0535,  0.0776, -0.3434, -0.2360, -0.4179,  0.2466,
        -0.3487])

In [8]:
f1_score([1,1,1,0,0,0,0,0,0], (torch.rand(9) - 0.5) > 0)

0.28571428571428575

In [4]:
from medvqa.metrics.classification.multilabel_f1score import f1_score

In [12]:
accuracy_score([0,0,0,1], [0,1,0,1])

0.75

In [19]:
(torch.rand(5, 10).detach() > 0) == 1

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

In [39]:
from sklearn.preprocessing import MultiLabelBinarizer

In [45]:
aux = [[1], [2,4]]

[[1], [2, 4]]

In [62]:
x = MultiLabelBinarizer(list(range(10))).fit_transform([[1], [0, 0, 0, 0, 5]])

In [63]:
x

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0]])

In [57]:
x

<2x10 sparse matrix of type '<class 'numpy.int64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [9]:
type("asfasdf") is list

False

In [1]:
from medvqa.datasets.mimiccxr.mimiccxr_vqa_dataset_management import MIMICCXR_VQA_Trainer

In [11]:
tokenizer = Tokenizer(qa_adapted_filenames=[iuxray_qa_adapted_reports_filename,
                                                mimiccxr_qa_adapted_reports_filename],
                          qa_adapted_datasets=[iuxray_qa_reports, mimiccxr_qa_reports],
                          min_freq=vocab_min_freq)
mvt = MIMICCXR_VQA_Trainer()

In [21]:
x = torch.rand([])
print(hasattr(x, '__len__'))
type(x) is torch.Tensor

True


True

In [22]:
import numbers

In [26]:
isinstance(x.item(), numbers.Number)

True

In [35]:
isinstance((x == x).item(), numbers.Number)

True

In [33]:
type(y)

int

In [18]:
x = {'asdf': 23, 'sdfsd':3, 'aaaa': 55, 'zzz':555}
vals = tuple(x[k] for k in sorted(list(x.keys())))
f'{vals}'

'(55, 23, 3, 555)'

In [21]:
from medvqa.utils.files import load_json_file
from medvqa.datasets.iuxray import IUXRAY_CACHE_DIR
import os

In [22]:
iuxray_reports = load_json_file(os.path.join(IUXRAY_CACHE_DIR, 'qa_adapted_reports__20220324_143133.json'))

In [24]:
iuxray_reports['questions'][69]

'pneumatocele?'